In [1]:
#Installation de Spark :
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
! wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
! tar xf /content/spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark
#Configuration de l'environnement pour utiliser Avro :
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages org.apache.spark:spark-avro_2.12:3.5.0 pyspark-shell"

#Création d'un objet SparkContext :
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")

from pyspark import SparkContext
sc = SparkContext.getOrCreate()
#Création d'un objet SparkSession
from pyspark.sql import SparkSession

# Redéfinissez SparkSession pour inclure le package Avro correct
# Création d'un objet SparkSession
spark = SparkSession.builder.master("local[*]").appName("TextClustering").getOrCreate()

In [2]:
#Téléchargement des données :
!wget http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz
#Décompression des données :
!tar -xzf 20news-19997.tar.gz
# Charger les données
all_documents = sc.wholeTextFiles("20_newsgroups/*")

--2024-02-04 16:57:45--  http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz
Resolving qwone.com (qwone.com)... 173.48.205.131
Connecting to qwone.com (qwone.com)|173.48.205.131|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17332201 (17M) [application/x-gzip]
Saving to: ‘20news-19997.tar.gz’

20news-19997.tar.gz 100%[===================>]  16.53M  7.72MB/s    in 2.1s    

2024-02-04 16:57:47 (7.72 MB/s) - ‘20news-19997.tar.gz’ saved [17332201/17332201]



In [13]:
all_documents.count()
all_documents.take(19997)[1996]

('file:/content/20_newsgroups/comp.graphics/38714',
 "Xref: cantaloupe.srv.cs.cmu.edu alt.graphics:557 alt.graphics.pixutils:5346 alt.internet.services:5110 comp.graphics:38714 comp.sys.sgi.graphics:909\nPath: cantaloupe.srv.cs.cmu.edu!rochester!udel!wupost!howland.reston.ans.net!agate!iat.holonet.net!UUCP!cboesel\nFrom: diablo.UUCP!cboesel (Charles Boesel)\nNewsgroups: alt.graphics,alt.graphics.pixutils,alt.internet.services,comp.graphics,comp.sys.sgi.graphics\nSubject: Is there an FTP achive for USGS terrain data\nDate: Wed, 21 Apr 93 17:49:35 PST\nOrganization: Diablo Creative\nMessage-ID: <0010580B.vmede1@diablo.UUCP>\nReply-To: diablo.UUCP!cboesel (Charles Boesel)\nX-Mailer: uAccess LITE - Macintosh Release: 1.6v2\nLines: 6\n\nIs there an FTP archive for United States Geological Services (USGS)\nterrain data? If so, where?\n\n--\ncharles boesel @ diablo creative |  If Pro = for   and   Con = against\ncboesel@diablo.uu.holonet.net    |  Then what's the opposite of Progress?\n+1.510

In [40]:
# Filtrer pour obtenir les documents de la catégorie "alt.atheism"
alt_atheism_rdd = all_documents.filter(lambda x: "alt.atheism" in x[0])
# Filtrer pour obtenir les documents de la catégorie "rec.sport.baseball"
rec_sport_baseball_rdd = all_documents.filter(lambda x: "rec.sport.baseball" in x[0])


In [41]:
rec_sport_baseball_rdd.count()
rec_sport_baseball_rdd.take(1000)[4]

('file:/content/20_newsgroups/rec.sport.baseball/102665',
 "Xref: cantaloupe.srv.cs.cmu.edu rec.sport.baseball:102665 alt.binaries.pictures.d:8635\nNewsgroups: rec.sport.baseball,alt.binaries.pictures.d\nPath: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!noc.near.net!uunet!newsflash.concordia.ca!nstn.ns.ca!cs.dal.ca!ug.cs.dal.ca!reiniger\nFrom: reiniger@ug.cs.dal.ca (Darren Reiniger)\nSubject: MLB logos: Part 3\nMessage-ID: <C52EIv.7yz@cs.dal.ca>\nSummary: National League East posted \nSender: usenet@cs.dal.ca (USENET News)\nNntp-Posting-Host: ug.cs.dal.ca\nOrganization: Math, Stats & CS, Dalhousie University, Halifax, NS, Canada\nDate: Tue, 6 Apr 1993 13:52:06 GMT\nLines: 10\n\n I have posted the logos of the NL East teams to alt.binaries.pictures.misc \n Hopefully, I'll finish the series up next week with the NL West.\n\n Darren\n\n-- \n Darren Reiniger                   reiniger@ug.cs.dal.ca || arishem@ac.dal.ca\n Ce

In [37]:
alt_atheism_rdd.count()
alt_atheism_rdd.take(1000)[2]

('file:/content/20_newsgroups/alt.atheism/53607',
 'Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:120958 alt.atheism:53607 talk.religion.misc:83953\nPath: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!zaphod.mps.ohio-state.edu!howland.reston.ans.net!agate!doc.ic.ac.uk!warwick!uknet!mcsun!Germany.EU.net!thoth.mchp.sni.de!horus.ap.mchp.sni.de!D012S658!frank\nFrom: frank@D012S658.uucp (Frank O\'Dwyer)\nNewsgroups: talk.abortion,alt.atheism,talk.religion.misc\nSubject: Re: After 2000 years, can we say that Christian Morality is\nDate: 23 Apr 1993 21:35:41 GMT\nOrganization: Siemens-Nixdorf AG\nLines: 79\nMessage-ID: <1r9nfd$3u4@horus.ap.mchp.sni.de>\nReferences: <sandvik-200493232227@sandvik-kent.apple.com> <1r39kh$itp@horus.ap.mchp.sni.de> <93112.164435J5J@psuvm.psu.edu>\nNNTP-Posting-Host: d012s658.ap.mchp.sni.de\n\nIn article <93112.164435J5J@psuvm.psu.edu> John A. Johnson <J5J@psuvm.psu.edu> writes:\n#In article <1r39kh$itp@horus.ap.mchp.sni.de

In [20]:
from pyspark.sql import Row

# Fonction pour séparer l'en-tête et le corps du message
def extract_header_body(text):
    parts = text.split("\n\n", 1)
    header = parts[0]
    body = parts[1] if len(parts) > 1 else None
    return header, body

# Fonction pour extraire les champs de l'en-tête
def extract_fields(header):
    fields = {"Organization": None, "Newsgroups": None}
    for line in header.split("\n"):
        if line.startswith("Organization:"):
            fields["Organization"] = line[len("Organization:"):].strip()
        elif line.startswith("Newsgroups:"):
            fields["Newsgroups"] = line[len("Newsgroups:"):].strip()
    return fields

# Transformation des RDDs et fusion
transformed_alt_atheism = alt_atheism_rdd.map(lambda x: extract_header_body(x[1])).map(lambda x: {**extract_fields(x[0]), "body": x[1]})
transformed_rec_sport_baseball = rec_sport_baseball_rdd.map(lambda x: extract_header_body(x[1])).map(lambda x: {**extract_fields(x[0]), "body": x[1]})


In [35]:
transformed_alt_atheism.count()
transformed_alt_atheism.take(1000)[1]

{'Organization': 'Digital Equipment Corporation',
 'Newsgroups': 'alt.atheism,talk.religion.misc,talk.origins',
 'body': '\nIn article <1993Apr15.225657.17804@rambo.atlanta.dg.com>, wpr@atlanta.dg.com \n\t(Bill Rawlins) writes:\n|>\n|>|> >|> \n|>|> >|> However, I have read a book:\n|>|> >|> \n|>|> >|>   GA Wells, _Did_Jesus_Exist_\n|>|> >|> \n|>|> >|>   ISBN 0301860025\n|>|> >|> \n|>|> >|> I found it to be thorough and interesting, but perhaps not the end of the\n|>|> >|> story.\n|>|> >|> \n|>|> >|> Perhaps you should read it and stop advancing the Bible as evidence relating \n|>|> >|> to questions of science.\n|>|> >\n|>|> >     There is a great fallacy in your statement. The question of origins is\n|>|> >     based on more than science alone.  If you met a man who could walk on\n|>|> >     water, raise people from the dead, claimed to be the Son of God, and\n|>|> >     then referred to the inviolability of the scriptures, this would affect\n|>|> >     your belief in the origin of man

In [31]:
# Conversion en RDD de Rows
row_rdd1 = transformed_alt_atheism.map(lambda x: Row(**x))

# Conversion en DataFrame
df = spark.createDataFrame(row_rdd1)

In [34]:
df.show(50)

+--------------------+--------------------+--------------------+
|        Organization|          Newsgroups|                body|
+--------------------+--------------------+--------------------+
|California Instit...|         alt.atheism|livesey@solntze.w...|
|Digital Equipment...|alt.atheism,talk....|\nIn article <199...|
|  Siemens-Nixdorf AG|talk.abortion,alt...|In article <93112...|
|Program in Evolut...|alt.atheism,talk....|In article <30147...|
|Okcforum Unix Use...|         alt.atheism|See, I told you t...|
|Penn State Univer...|         alt.atheism|jsn104 is jeremy ...|
|Technical Univers...|         alt.atheism|In article <66019...|
|The cat is on the...|         alt.atheism|On Tue, 20 Apr 19...|
|Sponsored account...|         alt.atheism|snm6394@ultb.isc....|
|                NCSU|         alt.atheism|Andrew Hilmer (hi...|
|Case Western Rese...|         alt.atheism|\n\n\tOur Lord an...|
|  Siemens-Nixdorf AG|talk.abortion,alt...|In article <sandv...|
|Case Western Rese...|   

In [21]:
transformed_rec_sport_baseball.count()
transformed_rec_sport_baseball.take(1000)[999]

{'Organization': 'Penn State University',
 'Newsgroups': 'rec.sport.baseball',
 'body': "Mike Schmidt's 500th:  Not only a milestone, but also a 9th inning game-\nwinner.\n                           -John\n"}

In [38]:
# Conversion en RDD de Rows
row_rdd2 = transformed_rec_sport_baseball.map(lambda x: Row(**x))

# Conversion en DataFrame
df = spark.createDataFrame(row_rdd2)

In [39]:
df.show(50)

+--------------------+--------------------+--------------------+
|        Organization|          Newsgroups|                body|
+--------------------+--------------------+--------------------+
|Homewood Academic...|  rec.sport.baseball|\tHello, my frien...|
|Netcom Online Com...|  rec.sport.baseball|In article <20APR...|
|Kansas State Univ...|  rec.sport.baseball|whitty@cv.hp.com ...|
|/usr/local/rn/org...|  rec.sport.baseball|In article <1quon...|
|Math, Stats & CS,...|rec.sport.basebal...| I have posted th...|
|University of Vic...|soc.culture.canad...|In article <93108...|
|Massachusetts Ins...|  rec.sport.baseball|In article <1qplh...|
|University of Hou...|  rec.sport.baseball|In article <1993A...|
|IBM AIX/ESA Devel...|  rec.sport.baseball|nflynn@wvnvms.wvn...|
|   Cray Research Inc|  rec.sport.baseball|In article <1993A...|
|Cornell Univ. CS ...|  rec.sport.baseball|In article <1qn6t...|
|Freshman, Mathema...|rec.sport.basebal...|>Does anybody kno...|
|University of Not...|  r

In [ ]:
# Conversion en RDD de Rows
row_rdd = combined_rdd.map(lambda x: Row(**x))

# Conversion en DataFrame
df = spark.createDataFrame(row_rdd)

# Sauvegarde au format Avro (nécessite le package Avro)
df.write.format("avro").save("messages.avro")

In [22]:
combined_rdd = transformed_alt_atheism.union(transformed_rec_sport_baseball)


In [24]:
combined_rdd.count()


2000

In [28]:
df.show(50)

+--------------------+--------------------+--------------------+
|        Organization|          Newsgroups|                body|
+--------------------+--------------------+--------------------+
|California Instit...|         alt.atheism|livesey@solntze.w...|
|Digital Equipment...|alt.atheism,talk....|\nIn article <199...|
|  Siemens-Nixdorf AG|talk.abortion,alt...|In article <93112...|
|Program in Evolut...|alt.atheism,talk....|In article <30147...|
|Okcforum Unix Use...|         alt.atheism|See, I told you t...|
|Penn State Univer...|         alt.atheism|jsn104 is jeremy ...|
|Technical Univers...|         alt.atheism|In article <66019...|
|The cat is on the...|         alt.atheism|On Tue, 20 Apr 19...|
|Sponsored account...|         alt.atheism|snm6394@ultb.isc....|
|                NCSU|         alt.atheism|Andrew Hilmer (hi...|
|Case Western Rese...|         alt.atheism|\n\n\tOur Lord an...|
|  Siemens-Nixdorf AG|talk.abortion,alt...|In article <sandv...|
|Case Western Rese...|   

In [49]:
# Conversion en RDD de Rows
row_rdd = combined_rdd.map(lambda x: Row(**x))

# Conversion en DataFrame
df = spark.createDataFrame(row_rdd)



In [50]:
from pyspark.sql.functions import countDistinct

# 1. Vérifier les catégories différentes de documents
df.select("Newsgroups").distinct().show()


# 2. Nombre d'organisations différentes
df.select(countDistinct("Organization")).show()

# 3. Autres statistiques descriptives
# Par exemple, le nombre de documents par catégorie
df.where((df["Newsgroups"] == "alt.atheism") | (df["Newsgroups"] == "rec.sport.baseball")).groupBy("Newsgroups").count().show()

# Si vous avez des champs numériques, vous pouvez calculer des statistiques comme la moyenne
# df.select(mean("VotreChampNumérique")).show()


+--------------------+
|          Newsgroups|
+--------------------+
|talk.religion.mis...|
|soc.culture.arabi...|
|soc.culture.arabi...|
|rec.scouting,soc....|
|talk.religion.mis...|
|alt.slack,talk.re...|
|alt.atheism,talk....|
|talk.abortion,alt...|
|talk.religion.mis...|
|alt.atheism,soc.m...|
|alt.atheism,soc.c...|
|alt.atheism,alt.f...|
|alt.atheism,alt.p...|
|alt.atheism,talk....|
|alt.atheism,rec.s...|
|alt.atheism,talk....|
|talk.abortion,alt...|
|alt.atheism,talk....|
|alt.atheism,talk....|
|alt.atheism,alt.a...|
+--------------------+
only showing top 20 rows

+----------------------------+
|count(DISTINCT Organization)|
+----------------------------+
|                         483|
+----------------------------+

+------------------+-----+
|        Newsgroups|count|
+------------------+-----+
|       alt.atheism|  734|
|rec.sport.baseball|  981|
+------------------+-----+



In [47]:
df.where((df["Newsgroups"] == "alt.atheism") | (df["Newsgroups"] == "rec.sport.baseball")).distinct().show()

+--------------------+-----------+--------------------+
|        Organization| Newsgroups|                body|
+--------------------+-----------+--------------------+
|California Instit...|alt.atheism|livesey@solntze.w...|
|Kupajava, East of...|alt.atheism|>DATE:   Mon, 5 A...|
|Technical Univers...|alt.atheism|In article <29441...|
|  Siemens-Nixdorf AG|alt.atheism|In article <C5r9A...|
|Decision Support ...|alt.atheism|In article <11654...|
|University of Oul...|alt.atheism|This kind of argu...|
|                 sgi|alt.atheism|In article <1993A...|
|                 sgi|alt.atheism|In article <1qjf3...|
|California Instit...|alt.atheism|livesey@solntze.w...|
|                 sgi|alt.atheism|In article <1ql5s...|
|University of Sou...|alt.atheism|bissda@saturn.wwc...|
|      VideOcart Inc.|alt.atheism|pl1u+@andrew.cmu....|
|Yale University, ...|alt.atheism|In article <kmr4....|
|University of Mar...|alt.atheism|In article <1993A...|
|University of Ill...|alt.atheism|In <lsran6INN1

In [51]:
from pyspark.ml.feature import Tokenizer, HashingTF

# 1. Découper les documents en listes de mots avec Tokenizer
tokenizer = Tokenizer(inputCol="body", outputCol="words")
wordsData = tokenizer.transform(df)

# 2. Créer une représentation vectorielle avec HashingTF
hashingTF = HashingTF(inputCol="words", outputCol="features")
featurizedData = hashingTF.transform(wordsData)

# Afficher les premières lignes pour vérifier
featurizedData.select("words", "features").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [52]:
from pyspark.ml.feature import IDF, Normalizer

# Supposons que 'featurizedData' est le résultat de votre HashingTF

# 4. Appliquer IDF pour obtenir des vecteurs TF-IDF
idf = IDF(inputCol="features", outputCol="tfidf")
idfModel = idf.fit(featurizedData)
tfidfData = idfModel.transform(featurizedData)

# 5. Normaliser les vecteurs
normalizer = Normalizer(inputCol="tfidf", outputCol="features_normalized", p=2.0)
normalizedData = normalizer.transform(tfidfData)

# Maintenant, 'normalizedData' est prêt à être utilisé pour KMeans ou d'autres algorithmes de clustering


In [53]:
from pyspark.ml.clustering import KMeans

# Supposons que 'normalizedData' est votre DataFrame contenant les vecteurs normalisés

# Configurer KMeans
kmeans = KMeans().setK(2).setSeed(1).setFeaturesCol("features_normalized")

# Entraîner le modèle KMeans
model = kmeans.fit(normalizedData)

# Prédire les clusters pour chaque document
predictions = model.transform(normalizedData)

# Afficher les résultats
predictions.select("prediction").show()


+----------+
|prediction|
+----------+
|         1|
|         1|
|         1|
|         1|
|         0|
|         0|
|         1|
|         1|
|         1|
|         0|
|         0|
|         1|
|         0|
|         1|
|         1|
|         1|
|         1|
|         0|
|         1|
|         1|
+----------+
only showing top 20 rows



In [56]:
from pyspark.sql.functions import monotonically_increasing_id

# Ajouter une colonne ID aux deux DataFrames
df_with_id = df.withColumn("id", monotonically_increasing_id())
predictions_with_id = predictions.withColumn("id", monotonically_increasing_id())

# Joindre les DataFrames sur la colonne ID
result_df = df_with_id.join(predictions_with_id, "id")

result_df.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|        Organization|          Newsgroups|                body|        Organization|          Newsgroups|                body|               words|            features|               tfidf| features_normalized|prediction|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 26|Okcforum Unix Use...|         alt.atheism|Kent Sandvik (san...|Okcforum Unix Use...|         alt.atheism|Kent Sandvik (san...|[kent, sandvik, (...|(262144,[2307,243...|(262144,[2307,243...|(262144,[2307,243...|         1|
| 29|  Siemens-Nixdorf AG|         alt.atheism|In article <16BB7...|  Siemens-Nixdorf AG|   